In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
path = 'csv/glassdoor_data_scientist_canada_221117_150.csv'

In [3]:
df = pd.read_csv(path)

In [4]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Division,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Junior Data Scientist,CA$74K - CA$90K (Glassdoor est.),We are looking for our next Junior Data Scient...,3.9,M&M Food Market\n3.9,NaN,Mississauga,1001 to 5000 Employees,NaN,Company - Private,Animal Production,Agriculture,$25 to $100 million (USD)
1,Junior Data Scientist,CA$11K (Employer est.),We are looking for our next Junior Data Scient...,3.9,M&M Food Market\n3.9,NaN,Mississauga,1001 to 5000 Employees,NaN,Company - Private,Animal Production,Agriculture,$25 to $100 million (USD)
2,Data Scientist – Revenue Management,CA$84K - CA$96K (Glassdoor est.),Powered by water... and by people like you\n\n...,-1.0,BC Hydro\n3.8,NaN,Vancouver,1001 to 5000 Employees,NaN,Self-employed,Energy & Utilities,"Energy, Mining & Utilities",Unknown / Non-Applicable
3,Data Scientist,CA$55K - CA$119K (Glassdoor est.),Join our Winning Team as a Data Scientist\nWhe...,4.5,Carfax\n4.5,NaN,London,1001 to 5000 Employees,1984.0,Company - Public,Internet & Web Services,Information Technology,Unknown / Non-Applicable
4,Analytics Data Scientist,CA$65K - CA$100K (Employer est.),BrainFinance is a leading financial technology...,-1.0,Brain Finance,NaN,Montreal,Unknown,NaN,Company - Private,NaN,NaN,Unknown / Non-Applicable


In [5]:
df.dtypes

Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Division             float64
Location              object
Size                  object
Founded              float64
Type of ownership     object
Industry              object
Sector                object
Revenue               object
dtype: object

In [6]:
def clean(path):
    
    df = pd.read_csv(path)
    
    df['Hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
    df['Employer Estimate'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer' in x.lower() else 0)

    df = df[df['Salary Estimate'] != '-1']
    salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])
    currency = df['Salary Estimate'].apply(lambda x: 'CAD' if 'CA' in x else None)
    no_signs = salary.apply(lambda x: re.sub('[^0-9,\-,-.]','', x))
    
    df['Salary Currency'] = currency
    
    df['min_salary'] = no_signs.apply(lambda x: float(x.split('-')[0]) if '-' in x else float(x))
    df['max_salary'] = no_signs.apply(lambda x: float(x.split('-')[1]) if '-' in x else float(x))
    df['avg_salary'] = (df.min_salary+df.max_salary)/2
    
    # Parse Company Name
    
    # Comment first line on new dataset as NaN values are set to -1
    df['Company Name'] = df['Company Name'].fillna('-1')
    df['Company'] = df['Company Name'].apply(lambda x: x.split('\n')[0])
    
#     #parsing of job description (python, etc.)
#     #python
#     df['python_yn'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)

#     #r studio 
#     df['R_yn'] = df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
#     df.R_yn.value_counts()

#     #spark 
#     df['spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
#     df.spark.value_counts()

#     #aws 
#     df['aws'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
#     df.aws.value_counts()

#     #excel
#     df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
#     df.excel.value_counts()

    df.columns

    new_path = path[:-4] + '_cleaned.csv'
    print('new path: ' + new_path)
    
    df_out = df#.drop(['Unnamed: 0'], axis =1)

    df_out.to_csv(new_path,index = False)
    return

In [7]:
clean(path)
df_cleaned = pd.read_csv(path[:-4] + '_cleaned.csv')
df_cleaned.head(50)

new path: csv/glassdoor_data_scientist_canada_221117_150_cleaned.csv


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Division,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,Hourly,Employer Estimate,Salary Currency,min_salary,max_salary,avg_salary,Company
0,Junior Data Scientist,CA$74K - CA$90K (Glassdoor est.),We are looking for our next Junior Data Scient...,3.9,M&M Food Market\n3.9,NaN,Mississauga,1001 to 5000 Employees,NaN,Company - Private,Animal Production,Agriculture,$25 to $100 million (USD),0,0,CAD,74.00,90.00,82.000,M&M Food Market
1,Junior Data Scientist,CA$11K (Employer est.),We are looking for our next Junior Data Scient...,3.9,M&M Food Market\n3.9,NaN,Mississauga,1001 to 5000 Employees,NaN,Company - Private,Animal Production,Agriculture,$25 to $100 million (USD),0,1,CAD,11.00,11.00,11.000,M&M Food Market
2,Data Scientist – Revenue Management,CA$84K - CA$96K (Glassdoor est.),Powered by water... and by people like you\n\n...,-1.0,BC Hydro\n3.8,NaN,Vancouver,1001 to 5000 Employees,NaN,Self-employed,Energy & Utilities,"Energy, Mining & Utilities",Unknown / Non-Applicable,0,0,CAD,84.00,96.00,90.000,BC Hydro
3,Data Scientist,CA$55K - CA$119K (Glassdoor est.),Join our Winning Team as a Data Scientist\nWhe...,4.5,Carfax\n4.5,NaN,London,1001 to 5000 Employees,1984.0,Company - Public,Internet & Web Services,Information Technology,Unknown / Non-Applicable,0,0,CAD,55.00,119.00,87.000,Carfax
4,Analytics Data Scientist,CA$65K - CA$100K (Employer est.),BrainFinance is a leading financial technology...,-1.0,Brain Finance,NaN,Montreal,Unknown,NaN,Company - Private,NaN,NaN,Unknown / Non-Applicable,0,1,CAD,65.00,100.00,82.500,Brain Finance
5,Data Scientist,CA$85K - CA$120K (Glassdoor est.),Referred applicants should not apply directly ...,3.6,Loblaw Companies Limited\n3.6,NaN,Brampton,10000+ Employees,NaN,Company - Public,Vehicle Dealers,Retail & Wholesale,$10+ billion (USD),0,0,CAD,85.00,120.00,102.500,Loblaw Companies Limited
6,"SENIOR MANAGER, DATA & ARTIFICIAL INTELLIGENCE...",CA$67K - CA$93K (Glassdoor est.),"Location: Pickering, ON, CA, L1W 3J2\nReq ID: ...",4.2,Ontario Power Generation\n4.2,NaN,Pickering,5001 to 10000 Employees,1999.0,Government,Energy & Utilities,"Energy, Mining & Utilities",$1 to $5 billion (USD),0,0,CAD,67.00,93.00,80.000,Ontario Power Generation
7,Lead Data Scientist (Hybrid),CA$56K - CA$108K (Glassdoor est.),"Job Category:\nProperty Tax, Software\nOpportu...",4.0,Altus Group\n4.0,NaN,Toronto,1001 to 5000 Employees,2005.0,Company - Public,Real Estate,Real Estate,$100 to $500 million (USD),0,0,CAD,56.00,108.00,82.000,Altus Group
8,Data Science and Analytics Manager,CA$76K - CA$88K (Glassdoor est.),Data Science Manager\nGALE is a creative media...,3.7,GALE Partners\n3.7,NaN,Toronto,501 to 1000 Employees,2014.0,Company - Private,Advertising & Public Relations,Media & Communication,Unknown / Non-Applicable,0,0,CAD,76.00,88.00,82.000,GALE Partners
9,Machine Learning Engineer,CA$60K - CA$98K (Glassdoor est.),Charger Logistics is a world class asset-based...,3.5,Charger Logistics Inc\n3.5,NaN,Brampton,1001 to 5000 Employees,2005.0,Company - Private,Taxi & Car Services,Transportation & Logistics,$5 to $25 million (USD),0,0,CAD,60.00,98.00,79.000,Charger Logistics Inc
